# Curs 4. Curatarea datelor - EIA

In [1]:
import pandas as pd

In [3]:
eia = pd.read_csv("../data/energy_transition_data.csv")


# 1) Profilare

In [ ]:
print("Coloanele disponibile:\n", eia.columns,"\nPrimele 2 randuri:")
eia.head(2)

Coloanele disponibile:
 Index(['period', 'productId', 'productName', 'activityId', 'activityName',
       'countryRegionId', 'countryRegionName', 'countryRegionTypeId',
       'countryRegionTypeName', 'dataFlagId', 'dataFlagDescription',
       'unitName', 'value', 'unit'],
      dtype='object') 
Primele 2 randuri


,period,productId,productName,activityId,activityName,countryRegionId,countryRegionName,countryRegionTypeId,countryRegionTypeName,dataFlagId,dataFlagDescription,unitName,value,unit
0,2024,4701,Gross domestic product,34,GDP,ABW,Aruba,c,Country,NaN,NaN,billion dollars at purchasing power parities,3.768053,BDOLPPP
1,2024,4701,Gross domestic product,34,GDP,AFG,Afghanistan,c,Country,NaN,NaN,billion dollars at purchasing power parities,64.92137,BDOLPPP


In [4]:
print("Dimensiune dataframe:", eia.shape)
print("\nTipuri de date:")
print(eia.dtypes)

Dimensiune dataframe: (357320, 14)

Tipuri de date:
period                     int64
productId                  int64
productName               object
activityId                 int64
activityName              object
countryRegionId           object
countryRegionName         object
countryRegionTypeId       object
countryRegionTypeName     object
dataFlagId               float64
dataFlagDescription       object
unitName                  object
value                     object
unit                      object
dtype: object


In [5]:
# Distribuție pe Country vs Region
print("\nTipuri de entități (Country vs Region):")
print(eia['countryRegionTypeName'].value_counts())


Tipuri de entități (Country vs Region):
countryRegionTypeName
Country    310000
Region      47320
Name: count, dtype: int64


In [6]:
eia['countryRegionTypeName'].value_counts()


countryRegionTypeName
Country    310000
Region      47320
Name: count, dtype: int64

In [7]:
eia['unit'].value_counts().head(20)


unit
QBTU       85425
MTOE       60831
TJ         60831
BKWH       59978
MT         24600
TST        24600
MMTCD      12199
MK         11964
THP         6359
BDOLPPP     6347
MST         4186
Name: count, dtype: int64

Unitati multiple
- QBTU       (Quadrillion BTU)
- MTOE       (Million tonnes oil equivalent)
- TJ         (Terajoules)
- BKWH       (Billion kWh)
- MT         (Thousand metric tons)
- TST        (Thousand short tons)
- MMTCD      (Million metric tons CO2)
- BDOLPPP    (Billion dollars PPP)


In [8]:
# Rate de lipsuri pe coloane
print("\nProcent lipsuri pe coloană (%):")
print((eia.isna().mean() * 100).round(2))


Procent lipsuri pe coloană (%):
period                    0.00
productId                 0.00
productName               0.00
activityId                0.00
activityName              0.00
countryRegionId           0.00
countryRegionName         0.00
countryRegionTypeId       0.00
countryRegionTypeName     0.00
dataFlagId               88.47
dataFlagDescription      88.47
unitName                  0.00
value                     0.40
unit                      0.00
dtype: float64


In [9]:
# Produse  (ex: Electricity, Coal, GDP)
print("\nTop 10 produse:")
print(eia['productName'].value_counts())


Top 10 produse:
productName
Coal                      145635
Electricity                95501
Primary energy             36852
Renewables                 30076
Natural gas                24353
Population                  6359
Gross domestic product      6347
CO2 emissions               6161
Oil                         6036
Name: count, dtype: int64


In [10]:
# Activități principale (Production, Consumption etc.)
print("\nTop 10 activități:")
print(eia['activityName'].value_counts().head(10))



Top 10 activități:
activityName
Consumption    67483
Production     61497
Imports        54874
Exports        54514
Generation     48187
Net imports    23672
Emissions      12199
Capacity       11964
Population      6359
GDP             6347
Name: count, dtype: int64


In [11]:
print(eia.productId.unique()) #produse unice dupa cod

[4701 4702    2    7   29   30   31   32   44 3002 4008 4411 4413]


In [12]:
# Ani disponibili
print("\nAni în dataset:")
print(eia['period'].unique()) 


Ani în dataset:
[2024 2023 2022 2021 2020 2019 2018 2017 2016 2015 2014 2013 2012 2011
 2010 2009 2008 2007 2006 2005 2004 2003 2002 2001]


### Toti indicatorii

In [13]:
eia[["productId", "productName", "activityName", "activityId"]].drop_duplicates().sort_values("productName")


,productId,productName,activityName,activityId
13681,4008,CO2 emissions,Emissions,8
9471,7,Coal,Reserves,6
14212,4411,Coal,Consumption,2
4203,7,Coal,Production,1
5520,7,Coal,Consumption,2
6837,7,Coal,Imports,3
8154,7,Coal,Exports,4
13947,4411,Coal,Production,1
11043,30,Coal,Generation,12
528,2,Electricity,Consumption,2


# 2). Standardizare

In [14]:
# facem modificarile pe o copie
eia_std = eia.copy()


In [15]:
# redenumim coloanele s
eia_std = eia_std.rename(columns={
    'period': 'year',
    'countryRegionId': 'country_iso3',
    'countryRegionName': 'country_name',
    'unitName': 'unit_name'
})


In [16]:
eia_std.columns

Index(['year', 'productId', 'productName', 'activityId', 'activityName',
       'country_iso3', 'country_name', 'countryRegionTypeId',
       'countryRegionTypeName', 'dataFlagId', 'dataFlagDescription',
       'unit_name', 'value', 'unit'],
      dtype='object')

In [17]:
# curatam textul
for col in ['productName', 'activityName', 'unit_name']:
    eia_std[col] = (
        eia_std[col]
            .astype(str)
            .str.strip()
            .str.lower()
            .str.replace(r'\s+', '_', regex=True)
    )


### Construim un nume pentru indicatori din produs si activitate - pentru nume, cod si label


#### Indicator name

In [18]:
# construire indicator_name (similar WDI)
eia_std['indicator_name'] = (
    eia_std['productName'] + '_' + eia_std['activityName']
)
eia_std.indicator_name.unique()

array(['gross_domestic_product_gdp', 'population_population',
       'electricity_consumption', 'electricity_imports',
       'electricity_exports', 'electricity_capacity',
       'electricity_distribution_losses', 'electricity_generation',
       'electricity_net_imports', 'coal_production', 'coal_consumption',
       'coal_imports', 'coal_exports', 'coal_reserves',
       'renewables_capacity', 'renewables_generation', 'coal_generation',
       'natural_gas_generation', 'oil_generation',
       'primary_energy_production', 'primary_energy_consumption',
       'natural_gas_emissions', 'co2_emissions_emissions',
       'natural_gas_production', 'natural_gas_consumption'], dtype=object)

In [19]:
# unele sunt dubluri - population_population
# Folosim .loc
eia_std.loc[
    eia_std['productName'] == eia_std['activityName'],
    'indicator_name'
] = eia_std['productName']

eia_std.indicator_name.unique()


array(['gross_domestic_product_gdp', 'population',
       'electricity_consumption', 'electricity_imports',
       'electricity_exports', 'electricity_capacity',
       'electricity_distribution_losses', 'electricity_generation',
       'electricity_net_imports', 'coal_production', 'coal_consumption',
       'coal_imports', 'coal_exports', 'coal_reserves',
       'renewables_capacity', 'renewables_generation', 'coal_generation',
       'natural_gas_generation', 'oil_generation',
       'primary_energy_production', 'primary_energy_consumption',
       'natural_gas_emissions', 'co2_emissions_emissions',
       'natural_gas_production', 'natural_gas_consumption'], dtype=object)

In [20]:
len(eia_std.indicator_name.unique())

25

#### Pentru indicator code

In [21]:
# construire indicator_code (consistent)
eia_std['indicator_code'] = (
    eia_std['productName'].str[:4].str.upper() + "_" +
    eia_std['activityName'].str[:3].str.upper()
)
eia_std['indicator_code'].unique()

array(['GROS_GDP', 'POPU_POP', 'ELEC_CON', 'ELEC_IMP', 'ELEC_EXP',
       'ELEC_CAP', 'ELEC_DIS', 'ELEC_GEN', 'ELEC_NET', 'COAL_PRO',
       'COAL_CON', 'COAL_IMP', 'COAL_EXP', 'COAL_RES', 'RENE_CAP',
       'RENE_GEN', 'COAL_GEN', 'NATU_GEN', 'OIL_GEN', 'PRIM_PRO',
       'PRIM_CON', 'NATU_EMI', 'CO2__EMI', 'NATU_PRO', 'NATU_CON'],
      dtype=object)

#### Pentru un indicator label

In [22]:
eia_std['indicator_label'] = (
    eia_std['productName'].str.replace('_', ' ').str.title()
    + " – " +
    eia_std['activityName'].str.replace('_', ' ').str.title()
)
eia_std.indicator_label.unique()

array(['Gross Domestic Product – Gdp', 'Population – Population',
       'Electricity – Consumption', 'Electricity – Imports',
       'Electricity – Exports', 'Electricity – Capacity',
       'Electricity – Distribution Losses', 'Electricity – Generation',
       'Electricity – Net Imports', 'Coal – Production',
       'Coal – Consumption', 'Coal – Imports', 'Coal – Exports',
       'Coal – Reserves', 'Renewables – Capacity',
       'Renewables – Generation', 'Coal – Generation',
       'Natural Gas – Generation', 'Oil – Generation',
       'Primary Energy – Production', 'Primary Energy – Consumption',
       'Natural Gas – Emissions', 'Co2 Emissions – Emissions',
       'Natural Gas – Production', 'Natural Gas – Consumption'],
      dtype=object)

#### Country type

In [23]:
# underscore
eia_std['countryRegionTypeName'] = eia_std['countryRegionTypeName'].str.lower()


In [24]:
eia_std['countryRegionTypeName'].unique()

array(['country', 'region'], dtype=object)

In [35]:
eia_std.columns

Index(['year', 'productId', 'productName', 'activityId', 'activityName',
       'country_iso3', 'country_name', 'countryRegionTypeId',
       'countryRegionTypeName', 'dataFlagId', 'dataFlagDescription',
       'unit_name', 'value', 'unit', 'indicator_name', 'indicator_code',
       'indicator_label'],
      dtype='object')

In [44]:
# extragem listele de tari si regiuni din EIA
countries_eia = eia_std[eia_std["countryRegionTypeName"] == "country"]["country_iso3"].unique()
regions_eia   = eia_std[eia_std["countryRegionTypeName"] == "region"]["country_iso3"].unique()

print("Număr țări EIA:", len(countries_eia))
print("Număr regiuni EIA:", len(regions_eia))

Număr țări EIA: 233
Număr regiuni EIA: 35


In [45]:
print(f"Primele 10 id-uri ale tarilor: {countries_eia[0:10]}")
print(f"Toate regiunile: {regions_eia}")

Primele 10 id-uri ale tarilor: ['ABW' 'AFG' 'AGO' 'ALB' 'ARE' 'ARG' 'ARM' 'ASM' 'ATA' 'ATG']
Toate regiunile: ['AFRC' 'ASOC' 'CSAM' 'EU27' 'EURA' 'EURO' 'MIDE' 'NOAM' 'NOEC' 'OEAO'
 'OECD' 'OEEU' 'OENA' 'OPAF' 'OPEC' 'OPNO' 'OPSA' 'PERG' 'WORL' 'WP11'
 'WP12' 'WP13' 'WP14' 'WP15' 'WP16' 'WP17' 'WP18' 'WP19' 'WP21' 'WP22'
 'WP23' 'WP24' 'WP25' 'WP26' 'WP27']


In [52]:
# Construim un dictionar simplu pentru maparea regiunilor

# filtrăm doar rândurile unde EIA spune clar că este "Region"
regions = eia_std[eia_std["countryRegionTypeName"] == "region"]
region_dict = dict(
    zip(regions["country_iso3"], regions["country_name"])
)

region_dict

{'AFRC': 'Africa',
 'ASOC': 'Asia & Oceania',
 'CSAM': 'Central & South America',
 'EU27': 'European Union',
 'EURA': 'Eurasia',
 'EURO': 'Europe',
 'MIDE': 'Middle East',
 'NOAM': 'North America',
 'NOEC': 'Non-OECD',
 'OEAO': 'OECD - Asia And Oceania',
 'OECD': 'OECD',
 'OEEU': 'OECD - Europe',
 'OENA': 'OECD - North America',
 'OPAF': 'OPEC - Africa',
 'OPEC': 'OPEC',
 'OPNO': 'Non-OPEC',
 'OPSA': 'OPEC - South America',
 'PERG': 'Persian Gulf',
 'WORL': 'World',
 'WP11': 'Australia and New Zealand',
 'WP12': 'IEO - Africa',
 'WP13': 'Brazil',
 'WP14': 'Canada',
 'WP15': 'China',
 'WP16': 'India',
 'WP17': 'Japan',
 'WP18': 'Mexico',
 'WP19': 'IEO - Middle East',
 'WP21': 'Western Europe',
 'WP22': 'Other Americas',
 'WP23': 'Other Asia-Pacific',
 'WP24': 'Eastern Europe and Eurasia',
 'WP25': 'Russia',
 'WP26': 'South Korea and other OECD Asia',
 'WP27': 'United States'}

In [54]:
# alternativ, putem folosi un dataframe pentru mapare
region_df = regions[["country_iso3", "country_name"]].drop_duplicates()
region_df.head()

,country_iso3,country_name
229,AFRC,Africa
230,ASOC,Asia & Oceania
231,CSAM,Central & South America
232,EU27,European Union
233,EURA,Eurasia


# PASUL 3: LIPSURI (Missing Analysis)

In [ ]:
#  Procent lipsuri pe fiecare coloană
print("Procent lipsuri pe coloană (%):")
print((eia_std.isna().mean() * 100).round(2))

Procent lipsuri pe coloană (%):
year                      0.00
productId                 0.00
productName               0.00
activityId                0.00
activityName              0.00
country_iso3              0.00
country_name              0.00
countryRegionTypeId       0.00
countryRegionTypeName     0.00
dataFlagId               88.47
dataFlagDescription      88.47
unit_name                 0.00
value                     0.40
unit                      0.00
indicator_name            0.00
indicator_code            0.00
indicator_label           0.00
dtype: float64


In [26]:
print("Numar de valori lipsa pentru coloana value")
print(eia_std.value.isna().sum())

Numar de valori lipsa pentru coloana value
1424


In [27]:
eia_std.dataFlagDescription.value_counts()

dataFlagDescription
A value exists, but rounds to zero; non-rounded values appear when downloaded    26642
The country does not exist during a given statistical time period                11300
Data are unavailable (permanently)                                                1424
Data are included elsewhere for this time period                                  1287
Values derived using SMR estimation methodology. Appears as a superscript.         497
Data row has been initialized                                                       36
Name: count, dtype: int64

In [28]:
eia_std.dataFlagId.value_counts()

dataFlagId
5.0     26642
3.0     11300
1.0      1424
4.0      1287
7.0       497
11.0       36
Name: count, dtype: int64

In [29]:
#eia_std['has_flag'] = eia_std['dataFlagId'].notna().astype(int)


In [30]:
missing_pct = (
    eia_std.groupby('dataFlagDescription')['value']
           .apply(lambda x: x.isna().mean())
           .sort_values(ascending=False)
)

print((missing_pct * 100).round(2))


dataFlagDescription
Data are unavailable (permanently)                                               100.0
A value exists, but rounds to zero; non-rounded values appear when downloaded      0.0
Data are included elsewhere for this time period                                   0.0
Data row has been initialized                                                      0.0
The country does not exist during a given statistical time period                  0.0
Values derived using SMR estimation methodology. Appears as a superscript.         0.0
Name: value, dtype: float64


In [31]:

# 5. Lipsuri pe fiecare indicator (optional, dar util didactic)
missing_by_indicator = (
    eia_std.groupby('indicator_name')['value']
           .apply(lambda x: x.isna().mean())
           .sort_values()
)

print("\nRate lipsuri pe indicator:")
print((missing_by_indicator * 100).round(2))


Rate lipsuri pe indicator:
indicator_name
co2_emissions_emissions            0.00
coal_consumption                   0.00
coal_exports                       0.00
coal_imports                       0.00
electricity_capacity               0.00
coal_production                    0.00
electricity_generation             0.00
electricity_consumption            0.00
electricity_net_imports            0.00
primary_energy_consumption         0.00
renewables_capacity                0.00
primary_energy_production          0.00
natural_gas_production             0.05
natural_gas_consumption            0.05
natural_gas_emissions              0.05
coal_generation                    0.20
natural_gas_generation             0.20
oil_generation                     0.20
coal_reserves                      0.36
renewables_generation              0.40
electricity_distribution_losses    0.53
electricity_imports                0.53
electricity_exports                0.54
population                         7.

In [33]:
#  Lipsuri pe ani
missing_by_year = (
    eia_std.groupby('year')['value']
           .apply(lambda x: x.isna().mean())
)
print("\nLipsuri pe an (%):")
print((missing_by_year * 100).round(2).head(20))


Lipsuri pe an (%):
year
2001    0.37
2002    0.35
2003    0.35
2004    0.35
2005    0.35
2006    0.34
2007    0.34
2008    0.33
2009    0.33
2010    0.33
2011    0.33
2012    0.32
2013    0.32
2014    0.32
2015    0.32
2016    0.32
2017    0.34
2018    0.53
2019    0.55
2020    0.57
Name: value, dtype: float64


# 4. Curatare

In [55]:
eia_country = eia_std[eia_std["countryRegionTypeName"] == "country"].copy()
eia_region = eia_std[eia_std["countryRegionTypeName"] == "region"].copy()
eia_region.to_csv("../data/eia_region.csv")

In [57]:
eia_country.columns

Index(['year', 'productId', 'productName', 'activityId', 'activityName',
       'country_iso3', 'country_name', 'countryRegionTypeId',
       'countryRegionTypeName', 'dataFlagId', 'dataFlagDescription',
       'unit_name', 'value', 'unit', 'indicator_name', 'indicator_code',
       'indicator_label'],
      dtype='object')

In [84]:
cols_to_drop = [
    'productId',
    'activityId',
    'countryRegionTypeId',
    'countryRegionTypeName',
    'dataFlagId',
    'dataFlagDescription',
    'productName',        # devine redundant
    'activityName'        # devine redundant
]

eia_clean = eia_country.drop(columns=cols_to_drop)


#### Dropna

In [70]:
# randuri goale
eia_clean = eia_clean.dropna(how='all')


#### Duplicate

In [86]:
eia_clean = eia_clean.drop_duplicates(keep='first')



In [87]:
eia_clean.value.isna().sum()

np.int64(1424)

# Transformare

In [94]:
eia_clean.columns

Index(['year', 'country_iso3', 'country_name', 'unit_name', 'value', 'unit',
       'indicator_name', 'indicator_code', 'indicator_label'],
      dtype='object')

In [88]:
eia_clean.head(2)

,year,country_iso3,country_name,unit_name,value,unit,indicator_name,indicator_code,indicator_label
0,2024,ABW,Aruba,billion_dollars_at_purchasing_power_parities,3.768053,BDOLPPP,gross_domestic_product_gdp,GROS_GDP,Gross Domestic Product – Gdp
1,2024,AFG,Afghanistan,billion_dollars_at_purchasing_power_parities,64.92137,BDOLPPP,gross_domestic_product_gdp,GROS_GDP,Gross Domestic Product – Gdp


In [97]:
# aranjam coloanele similar wdi

eia_clean= eia_clean[[
    'country_iso3',
    'country_name',
    'year',
    'indicator_code',
    'indicator_name',
    'indicator_label',
    'unit',
    'unit_name',
    'value'
]].copy()


In [98]:
eia_clean.head(2)

,country_iso3,country_name,year,indicator_code,indicator_name,indicator_label,unit,unit_name,value
0,ABW,Aruba,2024,GROS_GDP,gross_domestic_product_gdp,Gross Domestic Product – Gdp,BDOLPPP,billion_dollars_at_purchasing_power_parities,3.768053
1,AFG,Afghanistan,2024,GROS_GDP,gross_domestic_product_gdp,Gross Domestic Product – Gdp,BDOLPPP,billion_dollars_at_purchasing_power_parities,64.92137


## Salvam datele

In [100]:
eia_clean.to_csv("../data/eia_clean.csv", index = False)